In [86]:
import random
import time
import os

import cv2
import numpy as np

import torch
import torchvision
from PIL import Image, ImageFilter
from IPython import display

import torchvision.models
from torchvision.transforms import transforms

from tqdm.notebook import tqdm

In [51]:
def get_frame_sequence(directory, prefix, start, end):
    return [
        f"{directory}/{prefix}{frame_no:05d}.jpg" for frame_no in range(start, end + 1)
    ]

scene_frame_sequences = [
    get_frame_sequence("./frames/Train/Game", "Train_Game_01_", 1, 7),
    get_frame_sequence("./frames/Train/Game", "Train_Game_01_", 8, 119),
    get_frame_sequence("./frames/Train/Game", "Train_Game_01_", 120, 214),
    get_frame_sequence("./frames/Train/Game", "Train_Game_01_", 215, 302)
]

loaded_frame_sequences = [
    [Image.open(frame_loc) for frame_loc in scene_frame_sequence]
    for scene_frame_sequence in scene_frame_sequences
]

merged_frame_sequences = [(scene_frame_sequences[i][j], loaded_frame_sequences[i][j]) for i in range(len(scene_frame_sequences)) for j in range(len(scene_frame_sequences[i]))]
# merged_frame_sequences = [(frame_loc, frame) for frame_sequence in loaded_frame_sequences for frame in frame_sequence]

In [52]:
def score_frames(frame_1, frame_2):
    img_1 = np.array(frame_1, dtype=np.float64)
    img_2 = np.array(frame_2, dtype=np.float64)
    
    assert img_1.shape == img_2.shape
    
    mse = np.sum(np.sqrt(np.square(img_1 - img_2) ** 2)) / np.prod(img_1.shape)
    return mse
    
score_frames(loaded_frame_sequences[1][71], loaded_frame_sequences[1][72])

4.3217162181712965

In [55]:
def derive_sequences_with_test(images, threshold=500):
    sequences = []
    current_sequence = [images[0][1]]
    last_frame = current_sequence[0]
    
    for (current_frame_loc, current_frame) in images[1:]:
        consecutive_mse = score_frames(last_frame, current_frame)
        
        if consecutive_mse < threshold:
            current_sequence.append(current_frame)
        else:
            sequences.append(current_sequence)
            current_sequence = [current_frame]
            
            print("Broke at", current_frame_loc, "with score", consecutive_mse)
            
        last_frame = current_frame
            
    sequences.append(current_sequence)
    return sequences

In [56]:
seqs = derive_sequences_with_test(merged_frame_sequences)

Broke at ./frames/Train/Game/Train_Game_01_00008.jpg with score 2634.56570348669
Broke at ./frames/Train/Game/Train_Game_01_00120.jpg with score 2081.083927951389
Broke at ./frames/Train/Game/Train_Game_01_00215.jpg with score 2084.0212796585647


In [93]:
def derive_sequences_from_directory(ordered_dir, threshold, blur, save_dir=None):
    images = [f"{ordered_dir}/{file_name}" for file_name in os.listdir(ordered_dir) if file_name.endswith(".jpg")]
    
    if save_dir is not None:
        os.makedirs(save_dir)
    
    assert images == sorted(images)
    
    sequences = []
    current_sequence = [images[0]]
    last_frame = Image.open(images[0]).filter(ImageFilter.GaussianBlur(3))
    
    for current_frame_loc in tqdm(images[1:]):
        current_frame = Image.open(current_frame_loc).filter(ImageFilter.GaussianBlur(3))
        consecutive_mse = score_frames(last_frame, current_frame)
        
        if consecutive_mse < threshold:
            current_sequence.append(current_frame_loc)
        else:
            sequences.append(current_sequence)
            current_sequence = [current_frame_loc]
            
            if save_dir is not None:
                save_loc = f"{save_dir}/split_{len(sequences)}_{consecutive_mse}_{len(sequences[-1])}.jpg"
                frame_width, frame_height = current_frame.size
                split_showcase = Image.new("RGB", (frame_width * 2, frame_height))
                split_showcase.paste(last_frame, (0, 0))
                split_showcase.paste(current_frame, (frame_width, 0))
                split_showcase.save(save_loc)
            
            print("Broke at", current_frame_loc, "with score", consecutive_mse)
        
        last_frame = current_frame
        
    sequences.append(current_sequence)
    return sequences

In [94]:
threshold = 1500
blur = True
save_dir = f"./frame_split_testing/{time.time()}_{'blur' if blur else 'no_blur'}_{threshold}"
derived_sequences = derive_sequences_from_directory("./frames/Train/Game", threshold, blur, save_dir)

  0%|          | 0/5077 [00:00<?, ?it/s]

Broke at ./frames/Train/Game/Train_Game_01_00008.jpg with score 2522.4315483940973
Broke at ./frames/Train/Game/Train_Game_01_00120.jpg with score 1991.9615285011573
Broke at ./frames/Train/Game/Train_Game_01_00215.jpg with score 1960.8774996383102
Broke at ./frames/Train/Game/Train_Game_01_00303.jpg with score 2016.9306962528935
Broke at ./frames/Train/Game/Train_Game_01_00384.jpg with score 1988.8355711082177
Broke at ./frames/Train/Game/Train_Game_01_00424.jpg with score 1938.4114558015046
Broke at ./frames/Train/Game/Train_Game_01_00435.jpg with score 2569.925101634838
Broke at ./frames/Train/Game/Train_Game_01_00447.jpg with score 1867.4635970052084
Broke at ./frames/Train/Game/Train_Game_01_00453.jpg with score 1876.662536892361
Broke at ./frames/Train/Game/Train_Game_01_00458.jpg with score 1896.8105230034723
Broke at ./frames/Train/Game/Train_Game_01_00459.jpg with score 1930.6104097945602
Broke at ./frames/Train/Game/Train_Game_01_00502.jpg with score 2236.9138237847224
Broke 

KeyboardInterrupt: 